In [1]:
from utils import *
from imutils.video import VideoStream
import numpy as np
import argparse
import imutils
import time
import cv2
import os
import tensorflow as tf
from model import select_model, get_checkpoint

D:\anaconda3\envs\yolo\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\anaconda3\envs\yolo\lib\site-packages\tensorflow\python\framework\dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\anaconda3\envs\yolo\lib\site-packages\tensorflow\python\framework\dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\anaconda3\envs\yolo\lib\site-packages\tensorflow\python\framework\dtypes.py:522: FutureWarning: Passing (type

In [2]:
tf.app.flags.DEFINE_string('model_dir', '',
                           'Model directory (where training data lives)')

tf.app.flags.DEFINE_string('class_type', 'age',
                           'Classification type (age|gender)')


tf.app.flags.DEFINE_string('device_id', '/cpu:0',
                           'What processing unit to execute inference on')

tf.app.flags.DEFINE_string('filename', '',
                           'File (Image) or File list (Text/No header TSV) to process')

tf.app.flags.DEFINE_string('target', '',
                           'CSV file containing the filename processed along with best guess and score')

tf.app.flags.DEFINE_string('checkpoint', 'checkpoint',
                          'Checkpoint basename')

tf.app.flags.DEFINE_string('model_type', 'default',
                           'Type of convnet')

tf.app.flags.DEFINE_string('requested_step', '', 'Within the model directory, a requested step to restore e.g., 9000')

tf.app.flags.DEFINE_boolean('single_look', False, 'single look at the image or multiple crops')

tf.app.flags.DEFINE_string('face_detection_model', '', 'Do frontal face detection with model specified')

tf.app.flags.DEFINE_string('face_detection_type', 'cascade', 'Face detection model type (yolo_tiny|cascade)')

FLAGS = tf.app.flags.FLAGS

In [3]:
RESIZE_FINAL = 227
GENDER_LIST =['M','F']
AGE_LIST = ['(18,19)','(20,24)','(25,29)','(30,34)','(35,39)']
MAX_BATCH_SZ = 128

In [4]:
def getFaceBox(net, frame, conf_threshold=0.7):
    frameOpencvDnn = frame.copy()
    frameHeight = frameOpencvDnn.shape[0]
    frameWidth = frameOpencvDnn.shape[1]
    blob = cv2.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False)

    net.setInput(blob)
    detections = net.forward()
    bboxes = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            bboxes.append([x1, y1, x2, y2])
            cv2.rectangle(frameOpencvDnn, (x1, y1), (x2, y2), (0, 255, 0), int(round(frameHeight/150)), 8)
    return frameOpencvDnn, bboxes

In [30]:
def classify_one_multi_crop(sess, label_list, softmax_output, coder, images, image, writer):
    try:
        # 속도 개선
        image_batch = make_multi_crop_batch(image, coder)
        print('size of batches', image_batch.shape)
        print('session :', sess)
        batch_results = sess.run(softmax_output, feed_dict={images:image_batch.eval(session=sess)})
        output = batch_results[0]
        batch_sz = batch_results.shape[0]
        for i in range(1, batch_sz):
            output = output + batch_results[i]
        
        output /= batch_sz
        best = np.argmax(output)
        best_choice = (label_list[best], output[best])
        print('Guess @ 1 %s, prob = %.2f' % best_choice)
    
        if writer is not None:
            writer.writerow((best_choice[0], '%.2f' % best_choice[1]))
        return best_choice
    
    except Exception as e:
        print(e)
        print('Failed to run image')
        return '인식', '안됨'

In [6]:
faceProto = "opencv_face_detector.pbtxt"
faceModel = "opencv_face_detector_uint8.pb"
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)

In [7]:
# FACE RECOGNITION MODEL
faceNet = cv2.dnn.readNet(faceModel,faceProto)
coder = ImageCoder()

In [8]:
# AGE MODEL
age_graph = tf.Graph()
with age_graph.as_default():
    images = tf.placeholder(tf.float32, [None, RESIZE_FINAL, RESIZE_FINAL, 3])
    ageNet = select_model('default')
    logits = ageNet(len(AGE_LIST), images, 1, False)
    requested_step = 14999
    checkpoint_path = '%s' % ('C:/Users/LEE/Desktop/rude-carnie/age_model')
    model_checkpoint_path, global_step = get_checkpoint(checkpoint_path, requested_step, 'checkpoint')

selected default model
C:/Users/LEE/Desktop/rude-carnie/age_model/checkpoint-14999


In [9]:
# GENDER MODEL
gender_graph = tf.Graph()
with gender_graph.as_default():
    images2 = tf.placeholder(tf.float32, [None, RESIZE_FINAL, RESIZE_FINAL, 3])
    genderNet = select_model('default')
    logits2  = genderNet(len(GENDER_LIST), images2, 1, False)
    requested_step2 = 29999
    checkpoint_path2 = '%s' % ('C:/Users/LEE/Desktop/rude-carnie/gender_model')
    model_checkpoint_path2, global_step = get_checkpoint(checkpoint_path2, requested_step2, 'checkpoint')

selected default model
C:/Users/LEE/Desktop/rude-carnie/gender_model/checkpoint-29999


In [23]:
logits2

<tf.Tensor 'output/output:0' shape=(?, 2) dtype=float32>

In [24]:
age_sess = tf.Session(graph=age_graph) 
gender_sess = tf.Session(graph=gender_graph)

In [35]:
with age_sess.as_default():
    with age_graph.as_default():
        tf.global_variables_initializer().run()
        age_saver = tf.train.Saver(tf.global_variables())
        age_saver.restore(age_sess, model_checkpoint_path)

with gender_sess.as_default():
    with gender_graph.as_default():
        tf.global_variables_initializer().run()
        gender_saver = tf.train.Saver(tf.global_variables())
        gender_saver.restore(gender_sess, model_checkpoint_path2)  

INFO:tensorflow:Restoring parameters from C:/Users/LEE/Desktop/rude-carnie/age_model/checkpoint-14999
INFO:tensorflow:Restoring parameters from C:/Users/LEE/Desktop/rude-carnie/gender_model/checkpoint-29999


In [32]:
age_sess

In [13]:
gender_sess

In [14]:
'''sess = tf.Session()

# AGE MODEL
images = tf.placeholder(tf.float32, [None, RESIZE_FINAL, RESIZE_FINAL, 3])
ageNet = select_model('default')
logits = ageNet(len(AGE_LIST), images, 1, False)
requested_step = 14999
checkpoint_path = '%s' % ('C:/Users/LEE/Desktop/rude-carnie/age_model')
model_checkpoint_path, global_step = get_checkpoint(checkpoint_path, requested_step, 'checkpoint')

age_saver = tf.train.Saver()
age_saver.restore(sess, model_checkpoint_path)
'''

"sess = tf.Session()\n\n# AGE MODEL\nimages = tf.placeholder(tf.float32, [None, RESIZE_FINAL, RESIZE_FINAL, 3])\nageNet = select_model('default')\nlogits = ageNet(len(AGE_LIST), images, 1, False)\nrequested_step = 14999\ncheckpoint_path = '%s' % ('C:/Users/LEE/Desktop/rude-carnie/age_model')\nmodel_checkpoint_path, global_step = get_checkpoint(checkpoint_path, requested_step, 'checkpoint')\n\nage_saver = tf.train.Saver()\nage_saver.restore(sess, model_checkpoint_path)\n"

In [15]:
# Open a video file or an image file or a camera stream
cap = cv2.VideoCapture(1)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,800)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,800)
padding = 20

In [16]:
softmax_output2 = tf.nn.softmax(logits2)
softmax_output = tf.nn.softmax(logits)

In [39]:
while 1:
    # Read frame
    t = time.time()
    hasFrame, frame = cap.read()
    if not hasFrame:
        cv2.waitKey()
        break
    frameFace, bboxes = getFaceBox(faceNet, frame)
    
    if not bboxes:
        print("No face Detected, Checking next frame")
        continue

    for bbox in bboxes:
        face = frame[max(0,bbox[1]-padding):min(bbox[3]+padding,frame.shape[0]-1),max(0,bbox[0]-padding):min(bbox[2]+padding, frame.shape[1]-1)]
        wit
        h tf.device('/cpu:0'):
            writer = None
            output = None
            
            
            age,prob = classify_one_multi_crop(age_sess, AGE_LIST, softmax_output, coder, images, face, writer)
            #gender,prob = classify_one_multi_crop(gender_sess, GENDER_LIST, softmax_output2, coder, images2, face, writer)
            
            # 이 부분에서 이 에러가 납니다.
            # Cannot use the given session to evaluate tensor: the tensor's graph is different from the session's graph.
            
            gender = ''
        label = "{} {}".format(gender,age)
        cv2.putText(frameFace, label, (bbox[0]-5, bbox[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0,255), 2, cv2.LINE_AA)
        cv2.imshow("Age Gender Demo", frameFace)
    
    print("Time : {:.3f}".format(time.time() - t))
    key = cv2.waitKey(1) & 0xFF
    
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

# do a bit of cleanup
cv2.destroyAllWindows()

Running multi-cropped image
size of batches (12, 227, 227, 3)
session : <tensorflow.python.client.session.Session object at 0x000001B2CF8C8CF8>
Cannot use the given session to evaluate tensor: the tensor's graph is different from the session's graph.
Failed to run image
Time : 0.639
Running multi-cropped image
size of batches (12, 227, 227, 3)
session : <tensorflow.python.client.session.Session object at 0x000001B2CF8C8CF8>
Cannot use the given session to evaluate tensor: the tensor's graph is different from the session's graph.
Failed to run image
Time : 0.605
Running multi-cropped image
size of batches (12, 227, 227, 3)
session : <tensorflow.python.client.session.Session object at 0x000001B2CF8C8CF8>
Cannot use the given session to evaluate tensor: the tensor's graph is different from the session's graph.
Failed to run image
Time : 0.478
Running multi-cropped image
size of batches (12, 227, 227, 3)
session : <tensorflow.python.client.session.Session object at 0x000001B2CF8C8CF8>
Cann

error: OpenCV(4.3.0) C:\projects\opencv-python\opencv\modules\core\src\alloc.cpp:73: error: (-4:Insufficient memory) Failed to allocate 1080000 bytes in function 'cv::OutOfMemoryError'
